In [0]:
# Notebook: 02_model_training.py

import pyspark.sql.functions as F
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml import Pipeline

# 1. Definição da Tabela Bronze
BRONZE_TABLE = "dev_catalogue.staging_schema.bronze_insurance_costs"
SILVER_TABLE = "dev_catalogue.staging_schema.silver_insurance_features"

# Leitura da Tabela Delta
df_bronze = spark.read.table(BRONZE_TABLE)

print(f"DataFrame lido com sucesso da tabela Delta: {BRONZE_TABLE}")
df_bronze.show(5)

In [0]:
# Colunas Categóricas (features que precisam de encoding)
categorical_cols = ["sex", "smoker", "region"]

# Cria StringIndexers para as colunas categóricas
indexers = [
    StringIndexer(inputCol=col, outputCol=col + "_index", handleInvalid="keep")
    for col in categorical_cols
]

# Cria OneHotEncoders para as colunas indexadas
encoders = [
    OneHotEncoder(inputCol=col + "_index", outputCol=col + "_encoded", handleInvalid="keep")
    for col in categorical_cols
]

# Colunas Numéricas Originais (já prontas)
numerical_cols = ["age", "bmi", "children"]

# Colunas Finais Codificadas (as numéricas originais + as categóricas codificadas)
final_feature_cols = numerical_cols + [col + "_encoded" for col in categorical_cols]

# Cria o VectorAssembler para combinar todas as features em uma única coluna
vector_assembler = VectorAssembler(
    inputCols=final_feature_cols,
    outputCol="features"
)

# 3. Criação da Pipeline
pipeline = Pipeline(stages=indexers + encoders + [vector_assembler])

# Aplica a Pipeline no DataFrame
pipeline_model = pipeline.fit(df_bronze)
df_silver = pipeline_model.transform(df_bronze)

# Seleciona as colunas necessárias para o treinamento (features e charges)
df_silver = df_silver.select(
    F.col("charges").alias("label"), # Renomeia a variável alvo para 'label' (padrão do Spark ML)
    "features"
)

print("\nDataFrame SILVER (Pré-Processado) pronto:")
df_silver.show(5, truncate=False)

# 4. Salvar a Camada SILVER
df_silver.write.format("delta").mode("overwrite").saveAsTable(SILVER_TABLE)

print(f"\n✅ Camada SILVER salva com sucesso na Tabela Delta: {SILVER_TABLE}")


In [0]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

# 1. Divisão dos Dados (Treino e Teste)
# Usaremos 80% para treino e 20% para teste
(train_df, test_df) = df_silver.randomSplit([0.8, 0.2], seed=42)

print(f"\nDados de Treino: {train_df.count()} linhas")
print(f"Dados de Teste: {test_df.count()} linhas")

# 2. Treinamento do Modelo de Regressão Linear
lr = LinearRegression(featuresCol='features', labelCol='label', maxIter=10)
lr_model = lr.fit(train_df)

print("\nModelo de Regressão Linear treinado com sucesso.")

# 3. Avaliação no Conjunto de Teste
predictions_df = lr_model.transform(test_df)

# Avalia o modelo usando Root Mean Squared Error (RMSE)
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse"
)

rmse = evaluator.evaluate(predictions_df)
r2 = evaluator.evaluate(predictions_df, {evaluator.metricName: "r2"})

print("-" * 50)
print(f"Resultado da Avaliação do Modelo:")
print(f"  - RMSE (Root Mean Squared Error): {rmse:.2f}")
print(f"  - R2 (Coeficiente de Determinação): {r2:.4f}")
print("-" * 50)